# Core-Set

In [ ]:
first =[45.78, 44.66, 47.42, 43.27, 46.7]
second = [53.16, 53.0, 58.49, 55.0, 55.57]
third = [57.8, 58.02, 63.36, 62.74]
fourth = [65.31, 65.19, 67.5, 71.32]
fifth = [72.03, 73.1, 74.5, 72.97]


In [ ]:
first_mean = sum(first) / 5
second_mean = sum(second) / 5
third_mean = sum(third) / 4
fourth_mean = sum(fourth) / 4
fifth_mean = sum(fifth) / 4

In [ ]:
import matplotlib.pyplot as plt

plt.plot(["1K","2K","3K","4K","5K"], [first_mean,second_mean,third_mean, fourth_mean, fifth_mean])
plt.xlabel('number of Data (K)')
plt.ylabel('Accuracy')
plt.show()

# Learning Loss

In [ ]:
first_f1 = [[0.5020, 0.3265, 0.2425], [0.6170, 0.5261, 0.4990], [0.6665, 0.6912,0.6024], [0.7140, 0.7527,0.7039], [0.7754, 0.7607,0.7547]]
first_acc = [[0.5403, 0.3612,0.3313 ], [0.6418, 0.5582, 0.5164], [0.6776, 0.6985,0.6090], [0.7194, 0.7552,0.6985], [0.7701, 0.7672,0.7582]]

import matplotlib.pyplot as plt

lst = ["100","200","300","400","500"]
plt.plot(lst, [sum(first_f1[i])/3 for i in range(5)], label = 'F1_Score')
plt.plot(lst,  [sum(first_acc[i])/3 for i in range(5)],label = 'Acc')
plt.xlabel('number of Data')
plt.ylabel('Accuracy')
plt.title("Data | hz : 240 / bz : 240 / chem : 216 / yd : 124")
plt.legend()
plt.show()

# Classification

In [ ]:
first_f1 = [[0.5021],[0.5548],[],[],[]]
first_acc = [[0.5403],[0.5701],[],[],[]]

import matplotlib.pyplot as plt

lst = ["100","200","300","400","500"]
plt.plot(lst, [sum(first_f1[i]) for i in range(5)], label = 'F1_Score')
plt.plot(lst,  [sum(first_acc[i]) for i in range(5)],label = 'Acc')
plt.xlabel('number of Data')
plt.ylabel('Accuracy')
plt.title("Data | hz : 240 / bz : 240 / chem : 216 / yd : 124")
plt.legend()
plt.show()

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image

# Load the pretrained model
model = models.resnet18(pretrained=True)

# Use the model object to select the desired layer
layer = model._modules.get('avgpool')

# Set model to evaluation mode
model.eval()

# Image transforms
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

def get_vector(image_name):
    # 1. Load the image with Pillow library
    img = Image.open(image_name)
    # 2. Create a PyTorch Variable with the transformed image
    t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))
    # 3. Create a vector of zeros that will hold our feature vector
    #    The 'avgpool' layer has an output size of 512
    my_embedding = torch.zeros(512)
    # 4. Define a function that will copy the output of a layer
    def copy_data(m, i, o):
        my_embedding.copy_(o.data)
    # 5. Attach that function to our selected layer
    h = layer.register_forward_hook(copy_data)
    # 6. Run the model on our transformed image
    model(t_img)
    # 7. Detach our copy function from the layer
    h.remove()
    # 8. Return the feature vector
    return my_embedding.numpy()

img = "/Users/ibyeong-gwon/Desktop/fst/data/bz/raw_image/211201-AT6T31EH-M229_inpaint_1 ADR_W0.9_H1.3_BS.jpg"
print(get_vector(img))

/Users/ibyeong-gwon/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/ibyeong-gwon/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: output with shape [512] doesn't match the broadcast shape [1, 512, 1, 512]